<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [56]:
import pandas as pd
import numpy as np

In [57]:
train_for_test = pd.read_csv('train_for_test.csv')
train_for_test

,DateID,CategoryCode,ItemCode,Month,Day,Week,WeeklySales
0,2021-11-01,category_2,117610,11,6,w05,65
1,2022-01-24,category_1,370195,1,24,w17,37
2,2021-10-25,category_1,145978,10,30,w04,31
3,2022-01-24,category_1,1061341,1,24,w17,17
4,2021-12-06,category_1,1061341,12,7,w10,40
...,...,...,...,...,...,...,...
1694,2021-10-18,category_1,1081339,10,24,w03,3
1695,2021-10-04,category_1,371104,10,7,w01,6
1696,2021-11-15,category_1,37510,11,19,w07,3
1697,2021-10-18,category_2,57058,10,18,w03,4


In [58]:
sub = pd.read_csv('lstm_submission.csv')
sub

,ID,WeeklySales
0,category_1_43738_w4,22
1,category_2_1006090_w1,34
2,category_2_1076929_w4,80
3,category_1_1081321_w3,6
4,category_2_216151_w4,14
...,...,...
372,category_2_1101571_w1,64
373,category_2_1090258_w4,25
374,category_2_906595_w1,11
375,category_2_32245_w1,87


In [59]:
# s = sub['ID'].str.split('_')
# s

In [60]:
item = []
for s in sub['ID'].str.split('_'):
    item.append(int(s[2]))

In [61]:
sub['ItemCode'] = item

In [62]:
# train_for_test['ID'] = train_for_test['CategoryCode'] + '_' + train_for_test['ItemCode'].astype(str) + '_' + train_for_test['Week']

In [63]:
train_for_test.head()

,DateID,CategoryCode,ItemCode,Month,Day,Week,WeeklySales
0,2021-11-01,category_2,117610,11,6,w05,65
1,2022-01-24,category_1,370195,1,24,w17,37
2,2021-10-25,category_1,145978,10,30,w04,31
3,2022-01-24,category_1,1061341,1,24,w17,17
4,2021-12-06,category_1,1061341,12,7,w10,40


In [64]:
# train_for_test[train_for_test['ItemCode'] == 117610]['WeeklySales'].values.max()

In [65]:
sub[sub['ItemCode'] == 117610]['WeeklySales']

119     9
131     9
144    17
211    10
Name: WeeklySales, dtype: int64

In [66]:
sub[sub['ItemCode'] == 117610]['WeeklySales'] = [2,3,4,5]

C:\Users\Vipooshan\anaconda3\envs\tf\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [67]:
train_for_test[train_for_test['ItemCode'] == 1090258]['WeeklySales'].values

array([35, 18, 79, 21, 44,  2,  6,  5,  6,  7, 92, 38, 72,  4,  6, 19,  8,
        1], dtype=int64)

In [68]:
train_for_test[train_for_test['ItemCode'] == 1090258]['WeeklySales'].quantile([0.1,0.5,0.75, 0.9])

0.10     3.40
0.50    13.00
0.75    37.25
0.90    74.10
Name: WeeklySales, dtype: float64

In [69]:
train_for_test[train_for_test['ItemCode'] == 1090258]['WeeklySales'].max()

92

In [70]:
train_for_test[train_for_test['ItemCode'] == 1090258]['WeeklySales'].quantile([0.97])

0.97    85.37
Name: WeeklySales, dtype: float64

In [71]:
train_for_test[train_for_test['ItemCode'] == 1090258]['WeeklySales'].min()

1

In [79]:
train_for_test[train_for_test['ItemCode'] == 1090258]['WeeklySales'].quantile([0.1])

0.1    3.4
Name: WeeklySales, dtype: float64

In [73]:
sub[sub['ItemCode'] == 117610]['WeeklySales']

119     9
131     9
144    17
211    10
Name: WeeklySales, dtype: int64

In [74]:
sub[sub['ItemCode'] == 117610]['WeeklySales'].index.values

array([119, 131, 144, 211], dtype=int64)

In [75]:
sub.loc[sub[sub['ItemCode'] == 117610]['WeeklySales'].index, 'WeeklySales']

119     9
131     9
144    17
211    10
Name: WeeklySales, dtype: int64

In [76]:
np.where(sub[sub['ItemCode'] == 117610]['WeeklySales']>9, 3, sub[sub['ItemCode'] == 117610]['WeeklySales'])

array([9, 9, 3, 3], dtype=int64)

In [83]:
for i in sub['ItemCode'].values:
    item_max_sale = train_for_test[train_for_test['ItemCode'] == i]['WeeklySales'].quantile([0.97])
    item_max_sale = int(item_max_sale)
    indexes = sub[sub['ItemCode'] == i]['WeeklySales'].index.values
    
    change = np.where(sub[sub['ItemCode'] == i]['WeeklySales'] > item_max_sale, item_max_sale, sub[sub['ItemCode'] == i]['WeeklySales'])
#     print('before', sub[sub['ItemCode'] == i]['WeeklySales'][indexes])
    sub.loc[indexes, 'WeeklySales'] = change
#     print('after', sub[sub['ItemCode'] == i]['WeeklySales'][indexes])

In [84]:
for i in sub['ItemCode'].values:
    item_min_sale = train_for_test[train_for_test['ItemCode'] == i]['WeeklySales'].quantile([0.1])
    item_min_sale = int(item_min_sale)
    indexes = sub[sub['ItemCode'] == i]['WeeklySales'].index.values
    
    change = np.where(sub[sub['ItemCode'] == i]['WeeklySales'] < item_min_sale, item_min_sale, sub[sub['ItemCode'] == i]['WeeklySales'])
    
    sub.loc[indexes, 'WeeklySales'] = change

In [85]:
sub.tail()

,ID,WeeklySales,ItemCode
372,category_2_1101571_w1,64,1101571
373,category_2_1090258_w4,25,1090258
374,category_2_906595_w1,11,906595
375,category_2_32245_w1,51,32245
376,category_2_1006090_w2,28,1006090


In [86]:
sub.sort_values(by='WeeklySales', ascending=True)

,ID,WeeklySales,ItemCode
246,category_1_1068883_w4,1,1068883
337,category_2_1076929_w3,2,1076929
172,category_2_1101553_w2,3,1101553
31,category_2_1071115_w3,4,1071115
98,category_2_1070818_w4,4,1070818
...,...,...,...
288,category_4_123307_w3,574,123307
107,category_3_169504_w3,657,169504
161,category_4_123307_w4,659,123307
12,category_4_123307_w1,706,123307


In [87]:
sub[sub['ItemCode'] == 1076929]['WeeklySales']

2      60
20     24
155    12
337     2
Name: WeeklySales, dtype: int64

In [88]:
sub[sub['ItemCode'] == 1076929]['WeeklySales']

2      60
20     24
155    12
337     2
Name: WeeklySales, dtype: int64

In [89]:
train_for_test[train_for_test['ItemCode'] == 1076929]['WeeklySales'].values.min()

1

In [90]:
sub.drop(columns='ItemCode', inplace=True)

In [92]:
sub.to_csv('modified_96_with_bound_with_quartile.csv', index=False)